In [1]:
import torch 
import torchvision
import torch.nn as nn
import torch.utils.data as data
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

## Simple Example

In [2]:
# random normal
x = torch.randn(5, 3)
print (x)


 0.1479 -1.4306 -0.6625
-0.8837 -0.3919  0.4596
 1.0206  0.3912  0.4879
-1.2130 -2.0117 -0.7249
 0.8625  1.3113 -0.5966
[torch.FloatTensor of size 5x3]



In [3]:
# build a layer
linear = nn.Linear(3, 2)

In [4]:
# Sess weight and bias
print (linear.weight)
print (linear.bias)

Parameter containing:
 0.2596 -0.3333  0.2803
-0.1326 -0.0936 -0.2235
[torch.FloatTensor of size 2x3]

Parameter containing:
 0.0273
 0.3186
[torch.FloatTensor of size 2]



In [5]:
# forward propagate
y = linear(Variable(x))
print (y)

Variable containing:
 0.3568  0.5810
 0.0573  0.3698
 0.2986  0.0376
 0.1797  0.8299
-0.3530  0.2149
[torch.FloatTensor of size 5x2]



## Convert numpy array to torch tensor

In [6]:
# convert numpy array to tensor
a = np.array([[1,2], [3,4]])
b = torch.from_numpy(a)
print (b)


 1  2
 3  4
[torch.LongTensor of size 2x2]



## Input pipeline

#### (1) Preprocessing

In [7]:
# Image Preprocessing 
transform = transforms.Compose([
    transforms.Scale(40),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

### (2) Define Dataset

In [8]:
# download and loading dataset f
train_dataset = dsets.CIFAR10(root='./data/',
                               train=True, 
                               transform=transform,
                               download=True)

image, label = train_dataset[0]
print (image.size())
print (label)

Files already downloaded and verified
torch.Size([3, 32, 32])
6


### (3) Data Loader

In [9]:
# data loader provides queue and thread in a very simple way
train_loader = data.DataLoader(dataset=train_dataset,
                               batch_size=100, 
                               shuffle=True,
                               num_workers=2)

In [10]:
# iteration start then queue and thread start
data_iter = iter(train_loader)

# mini-batch images and labels
images, labels = data_iter.next()

for images, labels in train_loader:
    # your training code will be written here
    pass

### (4) What about custom dataset not cifar10?

In [11]:
class CustomDataset(data.Dataset):
    def __init__(self):
        lst_features = [[i*10+j for j in range(3)] for i in range(6)]
        self.features = torch.FloatTensor(lst_features)
        self.targets = torch.IntTensor([i for i in range(6)])
        
    # You should build the following two functions:
    # __len__: provides the size of the dataset,
    # __getitem__: return a sample data for given index
    
    def __getitem__(self, index):
        return self.features[index], self.targets[index]
    def __len__(self):
        return self.features.size(0)

In [12]:
custom_dataset = CustomDataset()
custom_loader = data.DataLoader(dataset=custom_dataset,
                                batch_size=3, 
                                shuffle=True,
                                num_workers=2)

In [13]:
for batch_index, (feature, target) in enumerate(custom_loader):
    print ('batch {}: {} {}'.format(batch_index, feature, target))

batch 0: 
 50  51  52
  0   1   2
 30  31  32
[torch.FloatTensor of size 3x3]
 
 5
 0
 3
[torch.LongTensor of size 3]

batch 1: 
 10  11  12
 20  21  22
 40  41  42
[torch.FloatTensor of size 3x3]
 
 1
 2
 4
[torch.LongTensor of size 3]



In [14]:
# You can also create an dataset by using data.TensorDataset
lst_features = [[i*10+j for j in range(3)] for i in range(6)]
features = torch.FloatTensor(lst_features)
targets = torch.IntTensor([i for i in range(6)])
my_data = data.TensorDataset(features, targets)
train_loader = data.DataLoader(my_data, batch_size=2, shuffle=True)

In [15]:
for batch_index, (feature, target) in enumerate(train_loader):
    print ('batch {}: {} {}'.format(batch_index, feature, target))

batch 0: 
 30  31  32
 50  51  52
[torch.FloatTensor of size 2x3]
 
 3
 5
[torch.LongTensor of size 2]

batch 1: 
 40  41  42
 10  11  12
[torch.FloatTensor of size 2x3]
 
 4
 1
[torch.LongTensor of size 2]

batch 2: 
 20  21  22
  0   1   2
[torch.FloatTensor of size 2x3]
 
 2
 0
[torch.LongTensor of size 2]



## Using Pretrained Model

In [16]:
# Download and load pretrained model
resnet = torchvision.models.resnet18(pretrained=True)

In [17]:
# delete top layer for finetuning
sub_model = nn.Sequential(*list(resnet.children())[:-1])

# for test
images = Variable(torch.randn(10, 3, 256, 256))
print (resnet(images).size())
print (sub_model(images).size())

torch.Size([10, 1000])
torch.Size([10, 512, 1, 1])


## Save and Load Model

In [18]:
# Save and load the trained model
torch.save(sub_model, 'model.pkl')

model = torch.load('model.pkl')